<a href="https://colab.research.google.com/github/Ish2276/VaR-CVaR-Model/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Download Historical Prices of the Portfolio Stocks

import yfinance as yf
import pandas as pd
import numpy as np

# Define the ticker symbols for the portfolio
tickers = ['ASTS', 'RKLB', 'LUNR','AAPL','META','GOOG']

# Download historical prices for the portfolio
data = yf.download(tickers, start='2022-01-01', end='2024-09-18')['Adj Close']

# Calculate daily returns for each stock
returns = data.pct_change().dropna()

# Preview the data
print(returns.head())

portfolio_value = 1_000_000  # Portfolio value in dollars (1 million)

[*********************100%***********************]  6 of 6 completed

Ticker                         AAPL      ASTS      GOOG      LUNR      META  \
Date                                                                          
2022-01-04 00:00:00+00:00 -0.012692 -0.034063 -0.004535  0.000000 -0.005937   
2022-01-05 00:00:00+00:00 -0.026600 -0.089421 -0.046830  0.000000 -0.036728   
2022-01-06 00:00:00+00:00 -0.016693  0.024896 -0.000745  0.000000  0.025573   
2022-01-07 00:00:00+00:00  0.000988  0.006748 -0.003973 -0.003096 -0.002015   
2022-01-10 00:00:00+00:00  0.000116 -0.054960  0.011456  0.000000 -0.011212   

Ticker                         RKLB  
Date                                 
2022-01-04 00:00:00+00:00 -0.045082  
2022-01-05 00:00:00+00:00 -0.081545  
2022-01-06 00:00:00+00:00 -0.004673  
2022-01-07 00:00:00+00:00  0.019718  
2022-01-10 00:00:00+00:00 -0.036832  


In [25]:
# Assign Portfolio Weights

from scipy.optimize import minimize

def portfolio_performance(weights, returns, risk_free_rate=0.0):
    # Calculate portfolio return
    portfolio_return = np.sum(returns.mean() * weights) * 252
    # Calculate portfolio standard deviation (risk)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    # Calculate Sharpe ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def negative_sharpe_ratio(weights, returns, risk_free_rate=0.0):
    # Minimize the negative Sharpe Ratio
    return -portfolio_performance(weights, returns, risk_free_rate)[2]

# Step 2: Set up optimization constraints and bounds
def optimize_portfolio(returns, risk_free_rate=0.0):
    num_assets = len(returns.columns)
    args = (returns, risk_free_rate)

    # Initial guess (equal weighting)
    initial_weights = num_assets * [1. / num_assets]

    # Constraints: weights must sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

    # Bounds: Weights can only be between 0 and 1 (long-only portfolio)
    bounds = tuple((0.1, 0.4) for asset in range(num_assets))

    # Optimize the portfolio by minimizing the negative Sharpe Ratio
    result = minimize(negative_sharpe_ratio, initial_weights, args=args, method='SLSQP', bounds=bounds, constraints=constraints)

    return result

# Step 3: Run optimization to get weights that maximize the Sharpe Ratio
optimal_result = optimize_portfolio(returns)
optimal_weights = optimal_result.x

# Step 4: Display the optimal weights and portfolio performance
portfolio_return, portfolio_std, sharpe_ratio = portfolio_performance(optimal_weights, returns)

print(f"Optimal Weights: {optimal_weights}")
print(f"Expected Portfolio Return: {portfolio_return:.2f}")
print(f"Expected Portfolio Risk (Standard Deviation): {portfolio_std:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Optimal Weights: [0.1        0.31494707 0.1        0.12513282 0.25992011 0.1       ]
Expected Portfolio Return: 0.58
Expected Portfolio Risk (Standard Deviation): 0.54
Sharpe Ratio: 1.07


In [9]:
# Compute the covariance matrix of the daily returns

cov_matrix = returns.cov()

# Print the covariance matrix
print("Covariance Matrix:\n", cov_matrix)

Covariance Matrix:
 Ticker      AAPL      ASTS      GOOG      LUNR      META      RKLB
Ticker                                                            
AAPL    0.000308  0.000352  0.000236 -0.000005  0.000282  0.000299
ASTS    0.000352  0.005663  0.000331  0.000306  0.000432  0.000985
GOOG    0.000236  0.000331  0.000440  0.000004  0.000404  0.000309
LUNR   -0.000005  0.000306  0.000004  0.015978 -0.000099  0.000034
META    0.000282  0.000432  0.000404 -0.000099  0.001016  0.000423
RKLB    0.000299  0.000985  0.000309  0.000034  0.000423  0.001811


In [26]:
# Calculate portfolio variance and standard deviation (volatility)
portfolio_variance = np.dot(optimal_weights, np.dot(cov_matrix, optimal_weights))
portfolio_volatility = np.sqrt(portfolio_variance)

print(f"Portfolio Volatility: {portfolio_volatility * 100:.2f}%")

Portfolio Volatility: 3.43%


In [11]:
# Calculate Portfolio VaR Using Variance-Covariance Method

from scipy.stats import norm

def portfolio_var(portfolio_volatility, confidence_level=0.95):
    # Use the inverse of the cumulative distribution function (ppf) of the normal distribution
    alpha = norm.ppf(1 - confidence_level)
    return portfolio_volatility * alpha

# Calculate 95% VaR using Variance-Covariance method
var_95 = portfolio_var(portfolio_volatility, 0.95)
var_95_value = var_95 * portfolio_value  # Scaling by portfolio value
print(f"95% Portfolio VaR (Variance-Covariance): ${var_95_value:.2f}")
print(f"95% Portfolio VaR (Variance-Covariance): {var_95 * 100:.2f}%")

95% Portfolio VaR (Variance-Covariance): $-63735.57
95% Portfolio VaR (Variance-Covariance): -6.37%


In [27]:
# Historical Simulation for Portfolio VaR

def historical_var_portfolio(returns, weights, confidence_level=0.95):
    # Calculate portfolio returns
    portfolio_returns = returns.dot(weights)

    # Sort the portfolio returns
    sorted_returns = portfolio_returns.sort_values()

    # Determine the VaR threshold index
    index = int((1 - confidence_level) * len(sorted_returns))

    # Return the VaR
    return sorted_returns.iloc[index]

# Calculate 95% VaR using Historical Simulation
var_historical_95 = historical_var_portfolio(returns,optimal_weights, 0.95)
var_Historical_95_value = var_historical_95 * portfolio_value  # Scaling by portfolio value
print(f"95% Portfolio VaR (Historical Simulation): ${var_Historical_95_value:.2f}")
print(f"95% Portfolio VaR (Historical Simulation): {var_historical_95 * 100:.2f}%")

95% Portfolio VaR (Historical Simulation): $-41967.62
95% Portfolio VaR (Historical Simulation): -4.20%


In [18]:
# Monte Carlo Simulation for Portfolio VaR

def monte_carlo_var_portfolio(returns, weights, num_simulations=10000, confidence_level=0.95):
    # Calculate portfolio mean and standard deviation
    portfolio_mean = np.dot(weights, returns.mean())
    portfolio_std = np.sqrt(np.dot(weights, np.dot(returns.cov(), weights)))

    # Generate random simulations
    simulated_returns = np.random.normal(portfolio_mean, portfolio_std, num_simulations)

    # Sort the simulated returns
    sorted_simulations = np.sort(simulated_returns)

    # Determine the VaR threshold index
    index = int((1 - confidence_level) * num_simulations)

    # Return the Monte Carlo VaR
    return sorted_simulations[index]

# Calculate 95% VaR using Monte Carlo Simulation
var_monte_carlo_95 = monte_carlo_var_portfolio(returns, optimal_weights, 10000, 0.95)
var_Monte_Carlo_95_value = var_monte_carlo_95 * portfolio_value  # Scaling by portfolio value
print(f"95% Portfolio VaR (Monte Carlo): ${var_Monte_Carlo_95_value:.2f}")
print(f"95% Portfolio VaR (Monte Carlo): {var_monte_carlo_95 * 100:.2f}%")

95% Portfolio VaR (Monte Carlo): $-61529.49
95% Portfolio VaR (Monte Carlo): -6.15%


In [28]:
# Calculate CVaR (Conditional Value at Risk) %
def cvar_portfolio(returns, weights, var):
    portfolio_returns = returns.dot(weights)
    return portfolio_returns[portfolio_returns <= var].mean()

cvar_historical_95 = cvar_portfolio(returns, optimal_weights, var_historical_95)
cvar_variance_covariance_95 = cvar_portfolio(returns, optimal_weights, var_95)
cvar_monte_carlo_95 = cvar_portfolio(returns, optimal_weights, var_monte_carlo_95)

print(f"95% Portfolio CVaR (Historical Simulation): {cvar_historical_95 * 100:.2f}%")
print(f"95% Portfolio CVaR (Variance-Covariance): {cvar_variance_covariance_95 * 100:.2f}%")
print(f"95% Portfolio CVaR (Monte Carlo): {cvar_monte_carlo_95 * 100:.2f}%")

95% Portfolio CVaR (Historical Simulation): -5.90%
95% Portfolio CVaR (Variance-Covariance): -7.77%
95% Portfolio CVaR (Monte Carlo): -7.77%


In [48]:
## CVaR Calculation based on historical simulation
def cvar_portfolio(returns, weights, var_threshold):
    # Calculate portfolio returns
    portfolio_returns = returns.dot(weights)

    # Filter out returns that are less than or equal to the VaR (losses beyond VaR)
    tail_losses = portfolio_returns[portfolio_returns <= var_threshold]

    # Calculate the average of those tail losses (this is the CVaR)
    if len(tail_losses) > 0:
        return tail_losses.mean()
    else:
        return 0  # Return zero if no losses exceed the VaR threshold

In [50]:
# CVaR Calculation based on  Monte Carlo
def monte_carlo_cvar_portfolio(returns, weights, var_threshold, num_simulations=10000):
    # Calculate portfolio mean and standard deviation
    portfolio_mean = np.dot(weights, returns.mean())
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))

    # Generate random simulations
    simulated_returns = np.random.normal(portfolio_mean, portfolio_std, num_simulations)

    # Filter out returns that are less than or equal to the VaR threshold
    tail_losses = simulated_returns[simulated_returns <= var_threshold]

    # Calculate CVaR as the average of those tail losses
    if len(tail_losses) > 0:
        return tail_losses.mean()
    else:
        return 0  # Return zero if no losses exceed the VaR threshold

In [51]:
#CVaR Calculation based on  Variance-Covariance
def variance_covariance_cvar_portfolio(portfolio_mean, portfolio_std, var_threshold):
    # Use the normal distribution assumption for portfolio returns
    simulated_returns = np.random.normal(portfolio_mean, portfolio_std, 10000)

    # Filter out returns that are less than or equal to the VaR threshold
    tail_losses = simulated_returns[simulated_returns <= var_threshold]

    # Calculate CVaR as the average of those tail losses
    if len(tail_losses) > 0:
        return tail_losses.mean()
    else:
        return 0  # Return zero if no losses exceed the VaR threshold

In [29]:
# prompt: # Calculate CVaR (Conditional Value at Risk) Portfolio Value

cvar_historical_95_value = cvar_historical_95 * portfolio_value
cvar_variance_covariance_95_value = cvar_variance_covariance_95 * portfolio_value
cvar_monte_carlo_95_value = cvar_monte_carlo_95 * portfolio_value

print(f"95% Portfolio CVaR (Historical Simulation): ${cvar_historical_95_value:.2f}")
print(f"95% Portfolio CVaR (Variance-Covariance): ${cvar_variance_covariance_95_value:.2f}")
print(f"95% Portfolio CVaR (Monte Carlo): ${cvar_monte_carlo_95_value:.2f}")


95% Portfolio CVaR (Historical Simulation): $-59020.48
95% Portfolio CVaR (Variance-Covariance): $-77741.08
95% Portfolio CVaR (Monte Carlo): $-77741.08


In [41]:
#Backtest VaR Model
def backtest_var(returns, weights, var_estimates, confidence_level=0.95):
    portfolio_returns = returns.dot(weights)
    actual_exceedances = portfolio_returns[portfolio_returns < -var_estimates].count()
    expected_exceedances = (1 - confidence_level) * len(portfolio_returns)

    print(f"Actual Exceedances: {actual_exceedances}")
    print(f"Expected Exceedances: {expected_exceedances}")

    if actual_exceedances > expected_exceedances:
        print("Warning: The VaR model underestimates the risk.")
    else:
        print("The VaR model performs well.")

backtest_var(returns, optimal_weights, var_historical_95)

Actual Exceedances: 622
Expected Exceedances: 33.95000000000003


In [49]:
# Stress Test the Portfolio with a price drop
def stress_test_price_drop(data, weights, shock_percentage=-0.1):
    # Apply a price drop to the stock prices
    stressed_prices = data * (1 + shock_percentage)

    # Recalculate returns from the stressed prices
    stressed_returns = stressed_prices.pct_change().dropna()

    # Recalculate VaR using historical simulation
    var_stressed_95 = historical_var_portfolio(stressed_returns, weights, 0.95)

    # Recalculate CVaR using the same historical simulation method
    cvar_stressed_95 = cvar_portfolio(stressed_returns, weights, var_stressed_95)

    print(f"95% Portfolio VaR after {shock_percentage*100:.1f}% price drop: {var_stressed_95 * 100:.2f}%")
    print(f"95% Portfolio CVaR after {shock_percentage*100:.1f}% price drop: {cvar_stressed_95 * 100:.2f}%")

# Example: Stress test with a 10% price drop
stress_test_price_drop(data, optimal_weights, shock_percentage=-0.1)

95% Portfolio VaR after -10.0% price drop: -4.20%
95% Portfolio CVaR after -10.0% price drop: -5.90%
